In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%watermark -v -p numpy,pandas,matplotlib,sklearn,torch,torchvision

CPython 3.6.9
IPython 7.16.1

numpy 1.18.5
pandas 1.0.4
matplotlib 3.2.1
sklearn 0.23.1
torch 1.6.0.dev20200609+cu101
torchvision 0.7.0.dev20200609+cu101


In [2]:
import warnings

import json
import os
from collections import OrderedDict
import numpy as np
import pandas as pd
import torch
import torchvision
from PIL import Image

import torch.nn.functional as F
from torch.optim import SGD, Adam
from torch.nn import Sequential, Module
from torch.nn import NLLLoss, MSELoss, CrossEntropyLoss
from torch.nn import BatchNorm2d, Conv2d, MaxPool2d, Dropout2d, Linear, ReLU, Flatten
from torch.optim.lr_scheduler import ExponentialLR, StepLR, MultiStepLR, ReduceLROnPlateau
from torch.utils.data import (Dataset, DataLoader)
from torchvision import transforms

warnings.filterwarnings('ignore')

from k12libs.utils.nb_easy import K12AI_DATASETS_ROOT
from k12libs.utils.nb_easy import (EasyaiClassifier, EasyaiTrainer, EasyaiDataset)

ImportError: cannot import name 'EasyaiClassifier'

---------------------------------------

## 简单实例

该实例没有实际意义, 默认配置的是 resnet18网络, 使用rmnist数据集

In [3]:
trainer = EasyaiTrainer(max_epochs=2, logger=False)
trainer.fit(EasyaiClassifier())
trainer.test()

NameError: name 'EasyaiTrainer' is not defined

----------------------------------------

## 接口

In [4]:
class CustomClassifier(EasyaiClassifier):

    ##########################################################################
    ####### Data ######
    ##########################################################################
    def load_presetting_dataset_(self, dataset_name):
        """
        加载平台预置的数据集
        Args:
            dataset_name: 数据集的名字, 如: rmnist, flowers等
        
        Return:
            以下几种方式任意一种:
            1. EasyaiDataset实例, 表明只进行训练(只返回了训练数据集实例)
            2. EasyaiDataset实例列表, 当列表长度为2时, 说明还要进行训练的校验, 当列表长度为3时, 说明还要进行测试评估.
            3. EasyaiDataset实例字典, 如: {'train': EasyaiDataset, 'val': EasyaiDataset, 'test':EasyaiDataset}
        """
        pass
    
    def prepare_dataset(self):
        """
        准备数据集, 从磁盘上加载数据集, 不同数据集的描述格式可能不一样, 一般有json/xml/csv等描述格式,
        也可能直接是图片目录, 所有这些格式的处理可以在这个接口完成.
            
        Return:
            同prepare_dataset
        """
        pass
    
    ##########################################################################
    ####### Model ######
    ##########################################################################
    def load_pretrained_model_(self, model_name, num_classes, pretrained):
        """
        加载平台预置的模型
        Args:
            model_name: 模型的名字
            num_classes: 分类个数(数据集labels的数目), 预置的模型默认是1000, 所以需要提供真实的分类个数进行处理
            pretrained: 是否加载预置权重
            
        Return:
            Module: 模型实例
        """
        pass
    
    def build_model(self):
        """
        构建模型, 可以自定义模型, 也可以调用load_pretrained_model_使用平台预置的模型
        
        Return:
            同load_pretrained_model_
        """
        pass
    
    
    ##########################################################################
    ####### Hypes Parameters ######
    ##########################################################################
    def configure_criterion(self):
        """
        配置损失函数
        
        Return:
            loss
        """
        pass
    
    def configure_optimizer(self):
        """
        配置优化器
        
        Return:
            optimizer
        """
        pass

    def configure_scheduler(self, optimizer):
        """
        配置学习率衰减策略
        
        Args:
            optimizer: 优化器(通过configure_optimizer配置得到的)
        
        Return:
            scheduler
        """
        pass
    
    def loss(self):
        """
        返回configure_criterion配置的损失实例
        """
        pass

    
    ##########################################################################
    ####### Trainer: Train ######
    ##########################################################################
    def train_dataloader(self):
        """
        训练数据集批量控制加载器, 可以设置批量的大小, 是否对数据进行洗牌(shuffle)等
        
        Return:
            loader
        """
        pass
    
    def training_step(self, batch, batch_idx):
        """
        训练过程中, 迭代一次batch数据, 就会触发一次training_step的调用,训练,统计metrics
        
        Args:
            batch: 一个batch的数据内容, 一般包括图片(image), 图片标签(labels), 图片路径(path).
                具体batch中内容受prepare_dataset接口的实现会有所不同
            batch_idx: 本轮epoch批量迭代次数
            
        Return:
            metrics: 必须包含loss关键字, log(日志模块)和progress_bar(进度条显示)是可选的
        """
        pass
    
    def training_epoch_end(self, outputs):
        """
        训练过程中, 每完整的遍历完一次(epoch)数据集,则触发一次training_epoch_end的调用, 汇总metrics
        
        Args:
            outputs: 所有metrics(training_step每次产生的数据的集合)
        
        Return:
            metrics: 同training_step, 可以对metrics做平均处理
        """
        pass

    ##########################################################################
    ####### Trainer: validation ######
    ##########################################################################
    def val_dataloader(self):
        """
        同train_dataloader
        """
        pass
    
    def validation_step(self, batch, batch_idx):
        """
        同train_step
        """
        pass
        
    def validation_epoch_end(self, outputs):
        """
        同train_epoch_end
        """
        pass
    
    ##########################################################################
    ####### Trainer: test ######
    ##########################################################################
    def test_dataloader(self):
        """
        同train_dataloader
        """
        pass

    def test_step(self, batch, batch_idx):
        """
        同train_step
        """
        pass
        
    def test_epoch_end(self, outputs):
        """
        同train_epoch_end
        """
        pass

NameError: name 'EasyaiClassifier' is not defined

## 实例: 预置数据集和预置模型

In [5]:
class CustomClassifier(EasyaiClassifier):
    def prepare_dataset(self):
        """
        加载预设数据集:chestxray (胸部xray判断新冠)
        """
        return self.load_presetting_dataset_(dataset_name='chestxray')

    def build_model(self):
        """
        加载预置模型: densenet201
        """
        return self.load_pretrained_model_(model_name='densenet201', num_classes=2, pretrained=True)

trainer = EasyaiTrainer(max_epochs=3, logger=False, progress_bar_rate=2)
# 训练
trainer.fit(CustomClassifier())
# 评估
trainer.test()

NameError: name 'EasyaiClassifier' is not defined

## 实例: 自定义损失函数,优化器, LR策略

In [6]:
class CustomClassifier(EasyaiClassifier):
    # 配置损失函数: CrossEntropyLoss(交叉熵损失)
    def configure_criterion(self):
        loss = CrossEntropyLoss(
            reduction='mean' # 约简方式: mean(张量各个维度上的元素的平均值)
        )
        return loss
    
    # 配置优化方法:
    def configure_optimizer(self):
        # self.model是在build_model构造的, 如果build_model没定义, 使用默认的构造的预置模型
        
        # optim-1: 随机梯度下降
        # optimizer = SGD(
        #     filter(lambda p: p.requires_grad, self.model.parameters()),
        #     lr=0.001,           # 基础学习率
        #     weight_decay=1e-6,  # 权重衰减, 使得模型参数值更小, 有效防止过拟合
        #     momentum=0.9,       # 动量因子, 更快局部收敛
        #     nesterov=False      # 使用Nesterov动量, 加快收敛速度
        # )
        
        # optim-2: 亚当
        optimizer = Adam(
            filter(lambda p: p.requires_grad, self.model.parameters()), # 过程出可更新的层(参数)
            lr=0.001,           # 基础学习率
            betas=(0.9, 0.999), # 计算梯度的均值(0.9)和平方(0.999)的系数
            eps=1e-8,           # 为了防止分母除零, 分母加上非常小的值
            weight_decay=0,     # 权重衰减
            amsgrad=False,      # 是否使用AmsGrad变体
        )
        return optimizer
    
    # 配置学习率策略: 
    def configure_scheduler(self, optimizer):
        # optmizer是在configure_optimizer配置的
        
        # policy-1: 指数衰减
        # scheduler = ExponentialLR(optimizer, gamma=0.5)
        
        # policy-2: 指标不在变化时, 调整学习率
        # scheduler = ReduceLROnPlateau(
        #     optimizer,   # 优化器
        #     mode='min',  # 指定指标不再下降
        #     factor=0.1,  # 衰减因子
        #     patience=6,  # 容忍多少次(指标不改变)
        #     eps=1e-6,    # 学习率衰减到的最小值eps时,学习率不再改变
        # )
        
        # policy-3: 固定步长衰减
        scheduler = StepLR(
            optimizer,   # 优化器
            step_size=2, # 每间隔2次epoch进行一次LR调整
            gamma=0.6    # LR调整为原来0.6倍
        )
        
        # policy-4: 不定步长衰减
        # scheduler = MultiStepLR(
        #     optimizer,        # 优化器
        #     milestones=[3, 7],# 分阶段(epoch)调整,到达指定的epoch时, 学习率调整为原来的gamma倍
        #     gamma=0.1         # LR调整为原来0.1倍
        # )
        return scheduler
    
trainer = EasyaiTrainer(max_epochs=2, logger=False)
trainer.fit(CustomClassifier())
trainer.test()

NameError: name 'EasyaiClassifier' is not defined

## 实例: 自定义训练阶段的Metrics

In [7]:
from k12libs.utils.nb_easy import (EasyaiClassifier, EasyaiTrainer)

class CustomClassifier(EasyaiClassifier):
    # 进入模型前, 数据加载参数设置
    def train_dataloader(self) -> DataLoader:
        return self.get_dataloader('train',
                batch_size=12,    # 一次进入模型的数据量(批量大小)
                num_workers=1,    # 启动多少个进程加载数据(值不要过大)
                drop_last=True,   # 是否丢掉最后一次不完整的batch(个数不足batch_szie)
                shuffle=False)    # 每次epoch时,是否将数据进行重新洗牌
    
    # 对每次batch迭代, 计算损失率, 正确率等, 返回metrics 
    def training_step(self, batch, batch_idx):
        x, y, p = batch           # images, labels, paths      
        y_hat = self.model(x)     # model由build_model返回, 输入images, 返回估计y
        loss = self.loss(y_hat, y)# loss有configure_criterion配置, 计算y_hat, y的差异损失
        with torch.no_grad():
            acc = (torch.argmax(y_hat, axis=1) == y).float().mean() # 计算正确率
            
        log = {'train_loss': loss, 'train_acc': acc}
        output = OrderedDict({
            'loss': loss,         # (M) 必须包含loss, 训练损失
            'acc': acc,           # (O) 可选, 每次迭代记录正确率, epoch结束时可以用来计算平均acc
            'progress_bar': log,  # (O) 可选, 可以在训练进度条上显示该字典里的内容
            "log": log            # (O) 可选, 如果启动logger模块, 作为logger的输入数据
        })
        return output

    # 对每次eopch的结束, 根据需要计算metrics, 输出日志等
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean() # 对训练loss做平均
        avg_acc = torch.stack([x['acc'] for x in outputs]).mean()   # 对训练acc做平均
        log = {'train_loss': avg_loss, 'train_acc': avg_acc}        # 封装到字典中
        return {'progress_bar': log, 'log': log}                    # 供进度条上和logger模块使用log字段数据
     
trainer = EasyaiTrainer(max_epochs=2, logger=False)
trainer.fit(CustomClassifier())

ImportError: cannot import name 'EasyaiClassifier'

## 实例: 自定义数据集(解析)及数据增强

In [17]:
class CustomClassifier(EasyaiClassifier):
    
    # 自定义数据集
    def prepare_dataset(self):
        # 继承EasyaiDataset, 实现data_reader接口
        class JsonfileDataset(EasyaiDataset):
            """
            Json文件描述的数据集的解析器
            """
            def data_reader(self, path, phase):
                """
                Args:
                    path: the dataset root directory
                    phase: the json file name (train.json / val.json / test.json)
                """
                image_list = []
                label_list = []
                with open(os.path.join(path, f'{phase}.json')) as f:
                    items = json.load(f)
                    for item in items:
                        image_list.append(os.path.join(path, item['image_path']))
                        label_list.append(item['label'])
                return image_list, label_list

        root = f'{K12AI_DATASETS_ROOT}/cv/rDogsVsCats' # 数据集路径
        mean = (0.4623, 0.4305, 0.2950)                # 数据集所有像素均值
        std  = (0.2520, 0.2242, 0.2091)                # 数据集所有像素的方差
        datasets = {
            'train': JsonfileDataset(mean=mean, std=std, path=root, phase='train'),
            'val':   JsonfileDataset(mean=mean, std=std, path=root, phase='val'),
            'test':  JsonfileDataset(mean=mean, std=std, path=root, phase='test'),
        }
        return datasets
    
    # 数据集标签不同时, 需要修改对应num_classes
    def build_model(self):
        return self.load_pretrained_model_('resnet18', 2)
    
    # 数据增强(场景:数据的样本比较小)
    def train_dataloader(self):
        """
        对训练集进行增强
        """
        trainloader = self.get_dataloader(
            phase='train',  # 训练数据集
            data_augment=[
                self.random_resized_crop(size=(128, 128)), # 随机对图片裁剪
                self.random_contrast(factor=0.25),         # 随机变换图片对比度
                self.random_brightness(factor=0.3),        # 随机变换图片的亮度
                self.random_rotation(degrees=30)           # 随机旋转图片(范围:-30-30)
            ],
            random_order=True,  # 变换方法执行顺序: 所有增强变换先随机排序在, 在依次变换
            input_size=128,     # 输入到模型的图片大小
            normalize=True,     # 是否对图片的Tensor数据进行归一化, 需要知道数据集像素的均值和方差
            batch_size=32,      # 一次输入到模型的批量大小
            num_workers=1,      # 启动多少进程加载数据(不建议设置过高)
            drop_last=False,    # 以后一个batch如果大小小于batch_size是否丢掉
            shuffle=False)      # 每一次epoch是否对样本(图片)数据进行随机洗牌(打散)
        
        return trainloader
    
    def val_dataloader(self):
        """
        对校验集进行增强
        """
        valloader = self.get_dataloader(
            phase='val',    # 校验数据集
            data_augment=[
                self.random_contrast(factor=0.25), # 随机变换图片对比度
                self.random_rotation(degrees=30)   # 随机旋转图片(范围:-30-30)
            ],
            random_order=False, 
            input_size=128,
            normalize=True,
            batch_size=16,
            num_workers=1,
            drop_last=False,
            shuffle=False)
        
        return valloader

trainer = EasyaiTrainer(max_epochs=2, logger=False)
trainer.fit(CustomClassifier())
trainer.test()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11 M  


--------------------------------------------------------------------------------
TEST RESULTS
{'test_acc': tensor(0.7850, device='cuda:0'),
 'test_loss': tensor(0.5356, device='cuda:0')}
--------------------------------------------------------------------------------



{'test_loss': 0.535595178604126, 'test_acc': 0.7849999666213989}

## 实例: 自定义网络模型

In [18]:
class CustomClassifier(EasyaiClassifier):
    def build_model(self):
        class SmallCNN(Module):
            """
            input_size: 128
            """
            def __init__(self, num_classes=10, hidden_size=100):
                super(SmallCNN, self).__init__()
                self.features = Sequential(
                    Conv2d(3, 32, 3, padding=1),ReLU(), # 128: (128-3+2*1)//1 + 1
                    Conv2d(32, 32, 3, padding=1, stride =1), ReLU(), # 128: (128-3+2*1)//1 + 1
                    Conv2d(32, 32, 3, padding=1, stride=2), ReLU(), # 64: (128-3+2*1)//2 + 1
                    Conv2d(32, 64, 3, padding=1), ReLU(),  # 64: (64-3+2*1)//1 + 1
                    Conv2d(64, 64, 3, padding=1, stride=2), ReLU()) # 32: (64-3+2*1)//2 + 1
                self.classifier = Sequential(
                    Flatten(), # 64*32*32
                    Linear(64*32*32, hidden_size), ReLU(),
                    Linear(hidden_size, num_classes))
    
            def forward(self, x):
                x = self.features(x)
                x = self.classifier(x)
                return x
            
        class CustomNet(Module):
            """
            input_size: 128
            """
            def __init__(self, num_classes=10):
                super(CustomNet, self).__init__()
                self.layer1 = Sequential(
                    Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
                    BatchNorm2d(16),
                    ReLU(),
                    MaxPool2d(kernel_size=2, stride=2))
                self.layer2 = Sequential(
                    Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
                    BatchNorm2d(32),
                    ReLU(),
                    MaxPool2d(kernel_size=2, stride=2))
                self.fc = Linear(32*32*32, num_classes)
                # 增加layer3
                # self.layer3 = Sequential(
                #     Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
                #     BatchNorm2d(64),
                #     ReLU(),
                #     MaxPool2d(kernel_size=2, stride=2))
                # self.fc = Linear(64*16*16, num_classes)
            
            def forward(self, x):
                x = self.layer1(x)
                x = self.layer2(x)
                # 增加layer3
                # x = self.layer3(x)
                x = x.reshape(x.size(0), -1)
                x = self.fc(x)
                return x
                
        # return SmallCNN(num_classes=10, hidden_size=50)
        return CustomNet(num_classes=10)
                    
    
trainer = EasyaiTrainer(
    logger=True,              # 启动tensorboard日志. 
    tb_port=9005,             # Tensorboard启动端口.
    max_epochs=20,            # 完整数据集迭代最大次数.
    model_summary_mode='full' # 查看模型的内存使用情况.
)

trainer.fit(CustomClassifier())
trainer.test()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


Selecting TensorBoard with logdir /data/tmp (started 0:00:00 ago; port 9005, pid 8790).



   | Name           | Type        | Params
------------------------------------------------
0  | model          | CustomNet   | 341 K 
1  | model.layer1   | Sequential  | 1 K   
2  | model.layer1.0 | Conv2d      | 1 K   
3  | model.layer1.1 | BatchNorm2d | 32    
4  | model.layer1.2 | ReLU        | 0     
5  | model.layer1.3 | MaxPool2d   | 0     
6  | model.layer2   | Sequential  | 12 K  
7  | model.layer2.0 | Conv2d      | 12 K  
8  | model.layer2.1 | BatchNorm2d | 64    
9  | model.layer2.2 | ReLU        | 0     
10 | model.layer2.3 | MaxPool2d   | 0     
11 | model.fc       | Linear      | 327 K 


--------------------------------------------------------------------------------
TEST RESULTS
{'test_acc': tensor(0.1017, device='cuda:0'),
 'test_loss': tensor(3.2203, device='cuda:0')}
--------------------------------------------------------------------------------



{'test_loss': 3.2203171253204346, 'test_acc': 0.1016865149140358}